# Dark Souls II Reviews (2025)

In [1]:
from get_data import *

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bdion\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bdion\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bdion\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bdion\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\bdion\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
reviews = get_data()

## Topic Modeling:
- Exploring certain aspects on why people like the game
    - Also get critiques of the game in positive reviews (if any but there sure is considering DS2's reputation in the community)

- Exploring why people don't like the game:
    - Also get positive aspects within this subset of the reviews

Splitting the reviews by how many do and don't recommend buying the game:

In [4]:
pos_reviews = reviews[reviews['voted_up'] == True]
neg_reviews = reviews[reviews['voted_up'] == False]

In [5]:
pos_reviews.shape, neg_reviews.shape

((37387, 10), (8176, 10))

### BERTopic:

In [6]:
from bertopic import BERTopic

In [ ]:
# topic_model = BERTopic(verbose=True)

In [8]:
docs = reviews.review.to_list()
pos_docs = pos_reviews.review.to_list()
neg_docs = neg_reviews.review.to_list()

In [ ]:
# topic_model.fit(docs)

# topic_model.save("models/bertopic_model")
topic_model = BERTopic.load("models/bertopic_model")

2025-04-06 10:36:44,667 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1424 [00:00<?, ?it/s]

2025-04-06 10:42:52,470 - BERTopic - Embedding - Completed ✓
2025-04-06 10:42:52,472 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-06 10:43:39,571 - BERTopic - Dimensionality - Completed ✓
2025-04-06 10:43:39,573 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-06 10:43:45,041 - BERTopic - Cluster - Completed ✓
2025-04-06 10:43:45,058 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-06 10:43:46,973 - BERTopic - Representation - Completed ✓
2025-04-06 10:43:49,433 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
c:\Users\bdion\anaconda3\Lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more effi

In [10]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,16198,-1_death_trash_play_rolin,"[death, trash, play, rolin, game, series, like...",[bad soulsborne huge margin even come close qu...
1,0,1531,0_mobile_forgot_ad_award,"[mobile, forgot, ad, award, show, till, commun...","[, , mobile ad]"
2,1,1239,1_attack_area_enemy_fight,"[attack, area, enemy, fight, player, bos, weap...",[oh god talk always tricky topic talk director...
3,2,1029,2_good_verry_uh_af,"[good, verry, uh, af, lol, thats, please, spoi...","[good, good, good]"
4,3,790,3_controller_keyboard_mouse_port,"[controller, keyboard, mouse, port, control, p...","[best enjoy keyboard mouse, good remember use ..."
...,...,...,...,...,...
634,633,10,633_underratted_claus_santa_bdsm,"[underratted, claus, santa, bdsm, fu, ii, aver...","[play ii favourite form bdsm, play game ii und..."
635,634,10,634_truly_actually_really_,"[truly, actually, really, , , , , , , ]","[truly, truly, truly]"
636,635,10,635_pill_boner_mom_worth,"[pill, boner, mom, worth, dollar, totally, way...","[great totally worth, almost hour safely say s..."
637,636,10,636_must_ape_cant_great,"[must, ape, cant, great, fantastic, wait, play...","[must play, great must play, great must play]"


In [11]:
topic_model = topic_model.reduce_topics(docs, nr_topics=30)
topic_model.visualize_topics()

2025-04-06 10:44:22,314 - BERTopic - Topic reduction - Reducing number of topics
2025-04-06 10:44:22,565 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-06 10:44:23,841 - BERTopic - Representation - Completed ✓
2025-04-06 10:44:23,850 - BERTopic - Topic reduction - Reduced number of topics from 639 to 30


In [12]:
topic_model.visualize_hierarchy()

### Positive reviews:

In [ ]:
# pos_topic_model = BERTopic(verbose=True)

In [ ]:
# pos_topic_model.fit(pos_docs)

# pos_topic_model.save("models/bertopic_pos_model")
pos_topic_model = BERTopic.load("models/bertopic_pos_model")

2025-04-06 10:44:37,589 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1169 [00:00<?, ?it/s]

2025-04-06 10:47:52,663 - BERTopic - Embedding - Completed ✓
2025-04-06 10:47:52,665 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-06 10:48:11,577 - BERTopic - Dimensionality - Completed ✓
2025-04-06 10:48:11,580 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-06 10:48:15,679 - BERTopic - Cluster - Completed ✓
2025-04-06 10:48:15,697 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-06 10:48:17,823 - BERTopic - Representation - Completed ✓
2025-04-06 10:48:20,917 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
c:\Users\bdion\anaconda3\Lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more eff

In [15]:
pos_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,13309,-1_death_play_game_love,"[death, play, game, love, series, recommend, g...",[feel like almost try one add weird mechanic c...
1,0,1253,0_show_till_as_even,"[show, till, as, even, , , , , , ]","[, , show]"
2,1,1005,1_good_uwu_mm_ahh,"[good, uwu, mm, ahh, af, lol, thats, dks, plea...","[good, good, good]"
3,2,823,2_area_enemy_attack_player,"[area, enemy, attack, player, fight, weapon, b...",[let preface review say lot fearmongering relu...
4,3,330,3_yes_yesn_yessir_bro,"[yes, yesn, yessir, bro, sir, , , , , ]","[yes, yes, yes]"
...,...,...,...,...,...
549,548,10,548_und_aber_wieder_spiel,"[und, aber, wieder, spiel, der, sich, wegen, n...",[good compare pros much well pvp connection in...
550,549,10,549_remake_development_remaster_cashgrab,"[remake, development, remaster, cashgrab, cons...",[please love kremmel unlikely scenario fromsof...
551,550,10,550_repeat_rinse_becouse_mantra,"[repeat, rinse, becouse, mantra, live, reitera...","[live die repeat, kill die repeat, die try rep..."
552,551,10,551_fan_complement_legitimately_collection,"[fan, complement, legitimately, collection, aw...",[legitimately fuck say also super good lot fun...


In [16]:
pos_topic_model = pos_topic_model.reduce_topics(pos_docs, nr_topics=30)
pos_topic_model.visualize_topics()

2025-04-06 10:48:31,860 - BERTopic - Topic reduction - Reducing number of topics
2025-04-06 10:48:32,093 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-06 10:48:33,538 - BERTopic - Representation - Completed ✓
2025-04-06 10:48:33,550 - BERTopic - Topic reduction - Reduced number of topics from 554 to 30


In [17]:
pos_topic_model.visualize_hierarchy()

### Negative reviews:

In [ ]:
# neg_topic_model = BERTopic(verbose=True)

In [ ]:
# neg_topic_model.fit(neg_docs)

# neg_topic_model.save("models/bertopic_neg_model")
neg_topic_model = BERTopic.load("models/bertopic_neg_model")

2025-04-06 10:48:34,334 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/256 [00:00<?, ?it/s]

2025-04-06 10:49:51,691 - BERTopic - Embedding - Completed ✓
2025-04-06 10:49:51,694 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-06 10:49:56,041 - BERTopic - Dimensionality - Completed ✓
2025-04-06 10:49:56,043 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-06 10:49:56,565 - BERTopic - Cluster - Completed ✓
2025-04-06 10:49:56,575 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-06 10:49:57,324 - BERTopic - Representation - Completed ✓
2025-04-06 10:49:58,137 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
c:\Users\bdion\anaconda3\Lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more eff

In [20]:
neg_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3466,-1_like_play_enemy_get,"[like, play, enemy, get, make, bad, good, game...",[spoiler ahead play twice anticipation year la...
1,0,306,0_enemy_player_bos_attack,"[enemy, player, bos, attack, area, fight, make...",[playthrough live reputation bad story engagin...
2,1,289,1_ii_xdddddddddddddddddddddddddddddddddddddddd...,"[ii, xdddddddddddddddddddddddddddddddddddddddd...",[xdddddddddddddddddddddddddddddddddddddddddddd...
3,2,112,2_bad_haha_sorry_yeah,"[bad, haha, sorry, yeah, somehow, still, say, ...","[bad, bad, bad]"
4,3,108,3_die_hard_enemy_hit,"[die, hard, enemy, hit, health, weapon, time, ...",[basic sometimes unfair always fun ii hold han...
...,...,...,...,...,...
132,131,11,131_controller_xbox_ps_setting,"[controller, xbox, ps, setting, pc, fucking, c...","[suck control xbox controller, recommend unles..."
133,132,11,132_review_negative_happiness_patch,"[review, negative, happiness, patch, positive,...",[browse negative review ii look good spend ren...
134,133,11,133_comparison_pretend_padded_ied,"[comparison, pretend, padded, ied, tr, wannabe...","[pal comparison every way, feel way wonky even..."
135,134,10,134_demon_covetous_covidtest_sme,"[demon, covetous, covidtest, sme, ter, sex, pl...","[bad bad demon, get demon, demon]"


In [21]:
neg_topic_model = neg_topic_model.reduce_topics(neg_docs, nr_topics=30)
neg_topic_model.visualize_topics()

2025-04-06 10:50:04,195 - BERTopic - Topic reduction - Reducing number of topics
2025-04-06 10:50:04,246 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-06 10:50:04,860 - BERTopic - Representation - Completed ✓
2025-04-06 10:50:04,864 - BERTopic - Topic reduction - Reduced number of topics from 137 to 30


In [22]:
neg_topic_model.visualize_hierarchy()